# BEHAVIOR Prep 2 — Mesh Basics (Vertices, Faces, Frames, Normals)
**Aim:** Treat meshes as geometry-in-frames (not graphics), and avoid common BEHAVIOR pitfalls.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
np.set_printoptions(precision=4, suppress=True)


In [ ]:
def skew(w):
    wx, wy, wz = w
    return np.array([[0, -wz, wy],
                     [wz, 0, -wx],
                     [-wy, wx, 0]], float)

def make_T(R, p):
    T = np.eye(4)
    T[:3,:3] = R
    T[:3,3] = p
    return T

def inv_T(T):
    R = T[:3,:3]
    p = T[:3,3]
    Rt = R.T
    return make_T(Rt, -Rt @ p)

def rotz(theta):
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[c,-s,0],[s,c,0],[0,0,1]], float)

def roty(theta):
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[c,0,s],[0,1,0],[-s,0,c]], float)

def plot_frame(ax, T, name="", L=0.12):
    o = T[:3,3]
    R = T[:3,:3]
    cols = ["r","g","b"]
    for i in range(3):
        ax.plot([o[0], o[0]+R[0,i]*L],
                [o[1], o[1]+R[1,i]*L],
                [o[2], o[2]+R[2,i]*L], c=cols[i], linewidth=2)
    if name:
        ax.text(o[0], o[1], o[2], " "+name)

def set_axes_equal(ax, lim=1.0):
    ax.set_xlim(-lim, lim)
    ax.set_ylim(-lim, lim)
    ax.set_zlim(-lim, lim)
    ax.set_xlabel("x"); ax.set_ylabel("y"); ax.set_zlabel("z")


## 1) Mesh = (V, F)
- `V`: vertices (N×3)
- `F`: triangle indices (M×3)

Vertices are expressed in a frame (typically the object frame).


In [ ]:
import numpy as np

V = np.array([
    [-0.5,-0.5,-0.5],
    [ 0.5,-0.5,-0.5],
    [ 0.5, 0.5,-0.5],
    [-0.5, 0.5,-0.5],
    [-0.5,-0.5, 0.5],
    [ 0.5,-0.5, 0.5],
    [ 0.5, 0.5, 0.5],
    [-0.5, 0.5, 0.5],
], float)

F = np.array([
    [0,1,2],[0,2,3],
    [4,6,5],[4,7,6],
    [0,4,5],[0,5,1],
    [1,5,6],[1,6,2],
    [2,6,7],[2,7,3],
    [3,7,4],[3,4,0],
], int)

print("V shape:", V.shape, "F shape:", F.shape)


## 2) Centroid vs AABB center


In [ ]:
centroid = V.mean(axis=0)
aabb_center = 0.5*(V.min(axis=0)+V.max(axis=0))
print("Centroid:", centroid)
print("AABB center:", aabb_center)


## 3) Face normal flips with winding


In [ ]:
def face_normal(v0, v1, v2):
    n = np.cross(v1-v0, v2-v0)
    return n/(np.linalg.norm(n)+1e-12)

tri = F[0]
n1 = face_normal(V[tri[0]], V[tri[1]], V[tri[2]])
n2 = face_normal(V[tri[0]], V[tri[2]], V[tri[1]])
print("normal:", n1)
print("flipped:", n2)


## 4) Quick visualization


In [ ]:
fig = plt.figure(figsize=(7,6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(V[:,0], V[:,1], V[:,2])

for tri in F[:6]:
    pts = V[tri]
    loop = np.vstack([pts, pts[0]])
    ax.plot(loop[:,0], loop[:,1], loop[:,2], linewidth=1)

plot_frame(ax, np.eye(4), "object", L=0.4)
set_axes_equal(ax, lim=1.0)
plt.title("Cube mesh in object frame")
plt.show()
